*This notebook was created by Sergey Tomin (sergey.tomin@desy.de). Source and license info is on [GitHub](https://github.com/ocelot-collab/ocelot). April 2025.*

# Ocelot for Students

This is a simple example designed to help students and beginners who are just starting to study accelerator physics.  
It aims to build intuition about how magnetic elements work.

## Duplet

In this section, we will consider a simple beamline consisting of only a drift and two quadrupoles.

As usual, let's import the necessary Ocelot modules, the standard Python `copy` library, and `ipywidgets` for interaction with the physics model.

> 💡 **Tip:** If you don't have the `ipywidgets` module installed, just search online for installation instructions:  
> `pip install ipywidgets`

In [1]:
import sys 
sys.path.append("/Users/tomins/Nextcloud/DESY/repository/ocelot/")

from ocelot import *
from ocelot.gui import *
from ipywidgets import interact, FloatSlider
import copy

initializing ocelot...


## Create a simple lattice

In [2]:
d = Drift(l=1.)
qf = Quadrupole(l=0.2, k1=1)
qd = Quadrupole(l=0.2, k1=-1)
cell = [d, qf, d, qd, d]

lat = MagneticLattice(cell)

## Let's Create a Simple Function to Plot Beta Functions

Now we define a function that calculates and plots the beta functions for a simple lattice with a focusing and defocusing quadrupole.  
You can interactively change the quadrupole strengths using sliders (if using `ipywidgets`), and observe how the beta functions evolve.

In [3]:
# Define a function that depends on some parameters
def plot_betas(qf_k1=1.0, qd_k1=-1.0):
    # Set quadrupole strengths
    qf.k1 = qf_k1
    qd.k1 = qd_k1

    # Define initial Twiss parameters
    tws0 = Twiss(beta_x=10, beta_y=10, alpha_x=0, alpha_y=0)

    # Calculate Twiss parameters along the beamline
    tws = twiss(lat, tws0, nPoints=20)

    # Extract s-position and beta functions
    sb = [tw.s for tw in tws]
    bx = [tw.beta_x for tw in tws]
    by = [tw.beta_y for tw in tws]

    # Plot using Ocelot's built-in function (you can also use matplotlib directly)
    fig, ax_xy = plot_API(lat, fig_name="Beta-functions", legend=False)
    ax_xy.plot(sb, bx, "C1", label=r"$\beta_x$")
    ax_xy.plot(sb, by, "C2", label=r"$\beta_y$")
    ax_xy.set_ylabel(r"$\beta_{x,y}$ [m]")
    ax_xy.set_xlabel("S [m]")
    ax_xy.legend()
    plt.show()

In [4]:
# Create interactive sliders for the parameters
interact(
    plot_betas,
    qf_k1=FloatSlider(min=-20, max=20, step=0.1, value=1.0),
    qd_k1=FloatSlider(min=-20, max=20, step=0.1, value=-1.0),
)

interactive(children=(FloatSlider(value=1.0, description='qf_k1', max=20.0, min=-20.0), FloatSlider(value=-1.0…

<function __main__.plot_betas(qf_k1=1.0, qd_k1=-1.0)>

## Plot Trajectories for Better Understanding of Transverse Beam Dynamics

Now let's plot particle trajectories with initial transverse offsets to better visualize how the beam evolves through the lattice.

We’ll use two subplots: one for horizontal (`x`) and one for vertical (`y`) motion.

In [5]:

def plot_beam(qf_k1=1.0, qd_k1=-1.0):
    qf.k1 = qf_k1
    qd.k1 = qd_k1

    # Create particles with transverse offsets
    x_coors = np.linspace(-0.01, 0.01, num=10)
    y_coors = np.linspace(-0.01, 0.01, num=10)

    p_list = [Particle(x=x, y=y) for x, y in zip(x_coors, y_coors)]
    P = [copy.deepcopy(p_list)]
    navi = Navigator(lat)
    dz = 0.01

    for _ in range(int(lat.totalLen / dz)):
        tracking_step(lat, p_list, dz=dz, navi=navi)
        P.append(copy.deepcopy(p_list))

    # Create subplots
    fig, (ax_x, ax_y) = plt.subplots(2, 1, figsize=(10, 6), sharex=True)
    fig.suptitle("Particle Trajectories with Transverse Offsets")

    for i in range(len(p_list)):
        s = [p[i].s for p in P]
        x = [p[i].x for p in P]
        y = [p[i].y for p in P]
        ax_x.plot(s, x, "C0")
        ax_y.plot(s, y, "C1")

    ax_x.set_ylabel("X [m]")
    ax_y.set_ylabel("Y [m]")
    ax_y.set_xlabel("S [m]")
    ax_x.grid(True)
    ax_y.grid(True)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

In [6]:
# Create interactive sliders for the parameters
interact(
    plot_beam,
    qf_k1=FloatSlider(min=-20, max=20, step=0.1, value=1.0),
    qd_k1=FloatSlider(min=-20, max=20, step=0.1, value=-1.0),
)

interactive(children=(FloatSlider(value=1.0, description='qf_k1', max=20.0, min=-20.0), FloatSlider(value=-1.0…

<function __main__.plot_beam(qf_k1=1.0, qd_k1=-1.0)>

## Bending Magnet: Quick Look at Dispersion

Let's take a look at transverse beam dynamics in a dipole magnet.

### A Simple Lattice

We define a basic beamline composed of a drift–dipole–drift sequence:


In [7]:
d = Drift(l=1)
b = Bend(l=0.5, angle=5/180*np.pi)
cell = [d, b, d]
lat2 = MagneticLattice(cell)

Now we’ll create particles with zero initial transverse coordinates, but different momentum deviations, and observe how this affects their horizontal position.
In other words, we’ll see dispersion in action 😊

In [8]:
def plot_beam_dipole(angle=0.01, dip_l=0.5):
    # Set dipole parameters (angle in degrees -> radians)
    b.angle = angle * np.pi / 180
    b.l = dip_l
    print(f"Lattice length: {lat2.totalLen:.2f} m")

    # Create particles with momentum deviations (x = 0, y = 0)
    p_coors = np.linspace(-0.01, 0.01, num=10)
    p_list = [Particle(x=0, y=0, p=p, E=1) for p in p_coors]
    P = [copy.deepcopy(p_list)]

    # Navigator and tracking step size
    navi = Navigator(lat2)
    dz = 0.01

    # Track particles through the lattice
    for _ in range(int(lat2.totalLen / dz)):
        tracking_step(lat2, p_list, dz=dz, navi=navi)
        P.append(copy.deepcopy(p_list))

    # Plot horizontal trajectories and momenta
    fig, (ax_x, ax_p) = plt.subplots(2, 1, figsize=(10, 6), sharex=True)
    fig.suptitle("Effect of Dispersion in a Dipole Magnet")

    for i in range(len(p_list)):
        s = [p[i].s for p in P]
        x = [p[i].x for p in P]
        p_vals = [p[i].p for p in P]

        ax_x.plot(s, x)
        ax_p.plot(s, p_vals, "C0")

    ax_x.set_ylabel("X [m]")
    ax_p.set_ylabel("Relative Momentum p")
    ax_p.set_xlabel("S [m]")
    ax_x.set_ylim([-0.0025, 0.0025])

    ax_x.grid(True)
    ax_p.grid(True)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

In [9]:
# Create interactive sliders for the parameters
interact(
    plot_beam_dipole,
    angle=FloatSlider(min=-10, max=10, step=0.05, value=1.0),
    dip_l=FloatSlider(min=0, max=2, step=0.1, value=1),
)

interactive(children=(FloatSlider(value=1.0, description='angle', max=10.0, min=-10.0, step=0.05), FloatSlider…

<function __main__.plot_beam_dipole(angle=0.01, dip_l=0.5)>